<a href="https://colab.research.google.com/github/iamamit118/deep-learning-1/blob/main/EXPERIMENT_3_DEEP_LEARNING_22BAI70169.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# Load and preprocess MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = X_train.reshape((-1, 28, 28, 1)), X_test.reshape((-1, 28, 28, 1))

# Convert labels to one-hot encoding for classification
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Define Autoencoder
input_layer = layers.Input(shape=(28, 28, 1))

# Encoder
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = models.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Pretrain Autoencoder (Unsupervised)
autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, shuffle=True)

# Extract Encoder for Greedy Pretraining
encoder = models.Model(input_layer, encoded)

# Fine-Tuning (Supervised Learning)
encoded_input = layers.Input(shape=(14, 14, 64))
x = layers.Flatten()(encoded_input)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

classifier = models.Model(encoded_input, x)
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Classifier with Encoded Features
classifier.fit(encoder.predict(X_train), y_train, epochs=5, batch_size=256, validation_data=(encoder.predict(X_test), y_test))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
 95/235 ━━━━━━━━━━━━━━━━━━━━ 1:47 767ms/step - loss: 0.3280

In [ ]:
import numpy as np

# Select some test images
n = 5
test_imgs = X_test[:n]
encoded_test_imgs = encoder.predict(test_imgs)

# Get classifier predictions
predictions = np.argmax(classifier.predict(encoded_test_imgs), axis=1)

# Plot results
plt.figure(figsize=(10, 2))
for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(test_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title(f"Pred: {predictions[i]}")
plt.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


NameError: name 'plt' is not defined

In [ ]:
\loss, accuracy = classifier.evaluate(encoder.predict(X_test), y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")